In [1]:
import sys
sys.path.append('../modules')
import likelihood_predictor
from likelihood_predictor import PlastPredictor
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.stats import zscore
import pickle
from sklearn.feature_selection import SelectFromModel

In [2]:
pl_full = pd.read_pickle('../database/plasticizer_data_v10_polarity.pkl')
pl_pol = pd.concat([pl_full[pl_full.columns[1:195]], pl_full['Polarity']], axis=1)
all_cols = pl_pol.columns.to_numpy()
pl_data = pl_pol[all_cols].to_numpy()
lin_data = pd.read_pickle('../database/linolein_test.pkl')
lin_data['Polarity'] = 0.048856
lin_data = lin_data[all_cols].to_numpy()
org_full = pd.read_pickle('../database/org_v3.pkl')

In [3]:
psim1 = open("pubs_similarity.txt", 'r')
psim11 = [line.rstrip('\n') for line in psim1]
psim2 = open("pubs_othersim.txt", 'r')
psim22 = [line.rstrip('\n') for line in psim2]

In [4]:
org_full
# org_full['Dsim'] = psim11
# org_full['Nasim'] = psim22
org_full = org_full.sort_values(by ='DEHP_sim')

In [5]:
org_full = org_full[:5000]
org_data = org_full[all_cols].to_numpy()

In [6]:
reg_param = 10
pp = PlastPredictor(reg_param)
pp_model = pp.fit_model(pl_data, org_data)


In [7]:
cc=pp.clf.coef_

In [8]:
np.count_nonzero(cc)

109

In [9]:
clf_file = 'clf_oneiteration.pkl'
scaler_file = 'scaler_oneiteration.pkl'
pp.save_model(clf_file, scaler_file)

In [14]:
org_full2 = pd.read_pickle('../database/org_v3.pkl')
org_data2 = org_full2[all_cols].to_numpy()

In [16]:
org_acc = pp.predict(org_data2, type='binary', class_id='neg')
pl_acc = pp.predict(pl_data, type='binary', class_id='pos')

In [17]:
org_acc, pl_acc

(0.7974374491350348, 0.9333333333333333)